In [1]:
import os
import pandas as pd
from astroquery.ipac.ned import Ned
from astropy.io import fits
from astropy.coordinates import SkyCoord
import astropy.units as u

In [11]:
# Load the 2MASS data from the CSV file
twomass_data = pd.read_csv('2mass.csv')

# Ensure object names match in both files (adjust the column name accordingly)
# Assuming 2MASS.csv has a column 'Name' that corresponds to the object names in the fits file
twomass_data.set_index('name', inplace=True)

# List of object names for redshift retrieval
object_names = [
    "1H0323+342", "1H1934-063", "3C120", "3C382", "3C390.3", "ARK120", "ARK564",
    "MCG-6-30-15", "MRK1018", "MRK110", "MRK279", "MRK335", "MRK509", "MRK590",
    "MRK79", "MRK841", "NGC1365", "NGC3783", "NGC4051", "NGC4151", "NGC4748",
    "NGC7469", "PG0804+761", "PG0844+349", "PG1426+015", "RBS1124", "UGC6728"
]

# Retrieve redshifts from NED
redshifts = {}
for object_name in object_names:
    try:
        result_table = Ned.query_object(object_name)
        redshift = result_table['Redshift'][0]
        redshifts[object_name] = redshift
    except Exception as e:
        redshifts[object_name] = 'N/A'
        print(f"Could not retrieve redshift for {object_name}: {e}")

# Function to parse RA and Dec from 'source' file
def parse_ra_dec(source_file):
    with open(source_file, 'r') as file:
        for line in file:
            if line.startswith('circle'):
                parts = line.split('(')[1].split(')')[0].split(',')
                ra = parts[0]
                dec = parts[1]
                return ra, dec
    return 'N/A', 'N/A'

# Function to convert RA and Dec to degrees
def convert_to_degrees(ra, dec):
    try:
        coord = SkyCoord(ra, dec, unit=(u.hourangle, u.deg))
        return round(coord.ra.degree, 4), round(coord.dec.degree, 4)
    except Exception as e:
        print(f"Could not convert RA/Dec to degrees for {ra}, {dec}: {e}")
        return 'N/A', 'N/A'

# Function to extract required information from a FITS file
def extract_fits_info(file_path, source_file):
    with fits.open(file_path) as hdul:
        header = hdul[0].header
        header_1 = hdul[1].header
        
        # Extracting the required information
        object_name = header.get('OBJECT', 'N/A')
        target_name = header.get('TARG_ID', 'N/A')
        obs_id = header.get('OBS_ID', 'N/A')
        filter_name = header.get('FILTER', 'N/A')
        exposure = header_1.get('EXPOSURE', 'N/A')
        date_obs = header.get('DATE-OBS', 'N/A')
        
        # Get redshift from the pre-fetched redshifts dictionary
        redshift = redshifts.get(object_name, 'N/A')
        
        # Get RA and Dec from the source file
        ra, dec = parse_ra_dec(source_file)
        
        # Convert RA and Dec to degrees
        ra_deg, dec_deg = convert_to_degrees(ra, dec)
        
        # Retrieve the corresponding 2MASS RA and Dec from the 2MASS CSV
        twomass_ra_deg = twomass_data.loc[object_name, 'ra'] if object_name in twomass_data.index else 'N/A'
        twomass_dec_deg = twomass_data.loc[object_name, 'dec'] if object_name in twomass_data.index else 'N/A'
        
        return {
            'Name': object_name,
            'Target ID': target_name,
            'Observation ID': obs_id,
            'Filter': filter_name,
            'Exposure Time (s)': round(exposure, 1),
            'Date/Time of Observation': date_obs,
            'RA (J2000)': ra,
            'Dec (J2000)': dec,
            'RA (deg)': round(ra_deg, 4),
            'Dec (deg)': round(dec_deg, 4),
            '2MASS RA (deg)': round(twomass_ra_deg, 4) if isinstance(twomass_ra_deg, (float, int)) else 'N/A',
            '2MASS Dec (deg)': round(twomass_dec_deg, 4) if isinstance(twomass_dec_deg, (float, int)) else 'N/A',
            'Redshift': redshift,
        }

# Base directory containing the subdirectories
base_directory = '/mnt/home/guptaagr/AGN-Photometry/swift27/'
directories = []

# Collect relevant subdirectories
for file in os.listdir(base_directory):
    filename = os.fsdecode(file)
    if filename[0] != '0':
        continue
    else:
        directories.append(filename)
directories = sorted(directories)

# List to hold the extracted information
data = []

# Loop through each subdirectory and process the FITS files
for directory in directories:
    fits_dir = os.path.join(base_directory, directory, 'uvot', 'image')
    source_file = os.path.join(fits_dir, 'source')
    if os.path.exists(fits_dir) and os.path.exists(source_file):
        for filename in os.listdir(fits_dir):
            if filename.endswith('sk.img.gz'):
                file_path = os.path.join(fits_dir, filename)
                info = extract_fits_info(file_path, source_file)
                data.append(info)

# Create a DataFrame from the extracted data
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('Tables/fits_info_table.csv', index=False)

print("Extraction complete. Data saved to 'fits_info_table.csv'.")

Extraction complete. Data saved to 'fits_info_table.csv'.
